In [24]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Función para cargar y preprocesar imágenes
def load_and_preprocess_images(data, img_size=(96, 96), path_col='path', label_col='label', filter_labels=None):
    images = []
    labels = []
    for index, row in data.iterrows():
        if filter_labels is None or row[label_col] in filter_labels:
            img_path = row[path_col].replace('..', 'data').replace('\\', '/')
            img = load_img(img_path, target_size=img_size, color_mode='rgb')  # Cambiado a 'rgb'
            img_array = img_to_array(img)
            images.append(img_array)
            labels.append(row[label_col])
    images = np.array(images, dtype='float32') / 255.0
    return images, labels

In [4]:
def load_and_preprocess_images2(data, img_size=(96, 96)):
    images = []
    for index, row in data.iterrows():
        img_path = f"data/data/images/test/{row['id_img']}.jpg"
        img = load_img(img_path, target_size=img_size, color_mode='rgb')  # Cambiado a 'rgb'
        img_array = img_to_array(img)
        images.append(img_array)
    images = np.array(images, dtype='float32') / 255.0
    return images

In [2]:
# Cargar los datos
train_data = pd.read_csv('data/train_set.csv')
test_data = pd.read_csv('test_set.csv')
sample_submission = pd.read_csv('sample_submision.csv')

In [5]:
test_images = load_and_preprocess_images2(test_data)

In [6]:
# Ajustar los nombres de las columnas según sea necesario
path_col = 'path'  # Cambia esto si el nombre de la columna es diferente
label_col = 'label'

In [7]:
# Mostrar las columnas de los DataFrames
print("Columnas de train_set.csv:", train_data.columns)
print("Columnas de test_set.csv:", test_data.columns)

Columnas de train_set.csv: Index(['id_img', 'path', 'label'], dtype='object')
Columnas de test_set.csv: Index(['id_img'], dtype='object')


In [8]:
train_data.label.value_counts()

label
happy       7164
neutral     4982
sad         4938
fear        4103
angry       3993
surprise    3205
disgust      436
Name: count, dtype: int64

In [9]:
# Cargar y procesar imágenes de entrenamiento
train_images, train_labels = load_and_preprocess_images(train_data, path_col=path_col, label_col=label_col, filter_labels=['angry', 'fear', 'happy', 'neutral', 'sad', 'surprise'])

In [10]:
# Codificar etiquetas para 7 clases
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
train_labels_categorical = to_categorical(train_labels_encoded, num_classes=6)

In [11]:
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels_categorical, test_size=0.2, random_state=42)

In [23]:
def create_optimized_cnn_model(input_shape, num_classes):
    model = Sequential()

    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(256, (3, 3), activation='relu', kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))

    model.add(Flatten())
    model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(BatchNormalization())

    model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(BatchNormalization())

    model.add(Dense(num_classes, activation='softmax'))

    return model


In [25]:
def lr_schedule(epoch, lr):
    if epoch > 20:
        lr = 0.0005
    if epoch > 40:
        lr = 0.0001
    return lr

In [16]:
input_shape = (96, 96, 3)  # Cambiado a 96x96 y color RGB
num_classes = 6
batch_size = 32
epochs = 120

In [17]:
model = create_cnn_model(input_shape, num_classes)

In [20]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Definir callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss')
lr_scheduler = LearningRateScheduler(lr_schedule)

In [22]:
history = model.fit(X_train, y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(X_val, y_val))

Epoch 1/25
710/710 ━━━━━━━━━━━━━━━━━━━━ 197s 277ms/step - accuracy: 0.4163 - loss: 1.4691 - val_accuracy: 0.4740 - val_loss: 1.3341
Epoch 2/25
710/710 ━━━━━━━━━━━━━━━━━━━━ 194s 273ms/step - accuracy: 0.4755 - loss: 1.3292 - val_accuracy: 0.5117 - val_loss: 1.2610
Epoch 3/25
710/710 ━━━━━━━━━━━━━━━━━━━━ 177s 249ms/step - accuracy: 0.5171 - loss: 1.2409 - val_accuracy: 0.5174 - val_loss: 1.2424
Epoch 4/25
710/710 ━━━━━━━━━━━━━━━━━━━━ 168s 237ms/step - accuracy: 0.5469 - loss: 1.1653 - val_accuracy: 0.5242 - val_loss: 1.2259
Epoch 5/25
710/710 ━━━━━━━━━━━━━━━━━━━━ 180s 254ms/step - accuracy: 0.5765 - loss: 1.1026 - val_accuracy: 0.5357 - val_loss: 1.2082
Epoch 6/25
710/710 ━━━━━━━━━━━━━━━━━━━━ 176s 248ms/step - accuracy: 0.5979 - loss: 1.0311 - val_accuracy: 0.5334 - val_loss: 1.2234
Epoch 7/25
710/710 ━━━━━━━━━━━━━━━━━━━━ 184s 260ms/step - accuracy: 0.6294 - loss: 0.9611 - val_accuracy: 0.5286 - val_loss: 1.2341
Epoch 8/25
710/710 ━━━━━━━━━━━━━━━━━━━━ 190s 267ms/step - accuracy: 0.6442 -

In [26]:
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

178/178 ━━━━━━━━━━━━━━━━━━━━ 10s 55ms/step - accuracy: 0.5406 - loss: 2.3878
Validation Accuracy: 54.24%


Fitting 3 folds for each of 108 candidates, totalling 324 fits
Mejores Parámetros para RandomForest: {'model__max_depth': 30, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 200}
Validation accuracy for RandomForest: 0.4062

In [34]:
test_predictions = model.predict(test_images)
test_predictions_labels = label_encoder.inverse_transform(test_predictions)

221/221 ━━━━━━━━━━━━━━━━━━━━ 12s 54ms/step


ValueError: y should be a 1d array, got an array of shape (7066, 6) instead.

In [28]:
submission = pd.DataFrame({'id_img': test_data['id_img'], 'label': test_predictions_labels})
submission.to_csv('submission.csv', index=False)

In [29]:
# Generar el archivo de submission con el nombre del modelo
submission_name = f"submission_{model}.csv"
submission = sample_submission.copy()

In [31]:
submission.to_csv(f'/data/submission/{submission_name}', index=False)

print(f"Submission file saved as {submission_name}")

OSError: Cannot save file into a non-existent directory: '\data\submission'

In [33]:
submission.label.value_counts()

label
surprise    7066
Name: count, dtype: int64